### Instalar librerías y reiniciar kernel.

In [1]:
# pip install mysql-connector
# pip install install pandas

### Crear acceso a engine de mysql e inspeccionar.

In [2]:
from sqlalchemy import create_engine

engine = create_engine('mysql+mysqlconnector://test:test123@db/test', echo = True)

In [3]:
import pandas as pd

pd.read_sql("select current_date", con=engine)

2021-02-03 03:00:53,473 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2021-02-03 03:00:53,475 INFO sqlalchemy.engine.base.Engine {}
2021-02-03 03:00:53,481 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2021-02-03 03:00:53,492 INFO sqlalchemy.engine.base.Engine {}
2021-02-03 03:00:53,499 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2021-02-03 03:00:53,500 INFO sqlalchemy.engine.base.Engine {}
2021-02-03 03:00:53,504 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2021-02-03 03:00:53,507 INFO sqlalchemy.engine.base.Engine {}
2021-02-03 03:00:53,511 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2021-02-03 03:00:53,512 INFO sqlalchemy.engine.base.Engine {}
2021-02-03 03:00:53,516 INFO sqlalchemy.engine.base.Engine DESCRIBE `select current_date`
2021-02-03 03:00:53,518 INFO sqlalchemy.engine.base.Engine {}
2021-02-03 03:00:53,521 INFO sqlalc

,current_date
0,2021-02-03


In [19]:
from sqlalchemy import inspect
inspector = inspect(engine)
schemas = inspector.get_schema_names()

for schema in schemas:
    print("schema: %s" % schema)
    for table_name in inspector.get_table_names(schema=schema):
        for column in inspector.get_columns(table_name, schema=schema):
            print("Column: %s" % column)

2021-02-03 03:07:09,736 INFO sqlalchemy.engine.base.Engine SHOW schemas
2021-02-03 03:07:09,737 INFO sqlalchemy.engine.base.Engine {}
schema: information_schema
2021-02-03 03:07:09,741 INFO sqlalchemy.engine.base.Engine SHOW FULL TABLES FROM `information_schema`
2021-02-03 03:07:09,742 INFO sqlalchemy.engine.base.Engine {}
schema: test
2021-02-03 03:07:09,753 INFO sqlalchemy.engine.base.Engine SHOW FULL TABLES FROM `test`
2021-02-03 03:07:09,755 INFO sqlalchemy.engine.base.Engine {}
2021-02-03 03:07:09,759 INFO sqlalchemy.engine.base.OptionEngine SHOW CREATE TABLE `test`.`vaccinations_per_country`
2021-02-03 03:07:09,761 INFO sqlalchemy.engine.base.OptionEngine {}
Column: {'name': 'index', 'type': BIGINT(display_width=20), 'default': None, 'comment': None, 'nullable': True, 'autoincrement': False}
Column: {'name': 'country', 'type': TEXT(), 'default': None, 'comment': None, 'nullable': True}
Column: {'name': 'iso_code', 'type': TEXT(), 'default': None, 'comment': None, 'nullable': True

In [18]:
print(engine.table_names())

2021-02-03 03:06:58,013 INFO sqlalchemy.engine.base.Engine SHOW FULL TABLES FROM `test`
2021-02-03 03:06:58,015 INFO sqlalchemy.engine.base.Engine {}
['vaccinations_per_country']


# ETL 1

### Extraer data del source y explorar.

In [7]:
df = pd.read_csv('country_vaccinations.csv', 
                 dtype={"total_vaccinations": "Int64", 
                        "people_vaccinated": "Int64", 
                        "people_fully_vaccinated": "Int64", 
                        "daily_vaccinations": "Int64", 
                        "daily_vaccinations_per_million": "Int64"}, encoding='utf-8')
df.head()

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Argentina,ARG,2020-12-29,700,<NA>,<NA>,NaN,<NA>,0.00,NaN,NaN,<NA>,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
1,Argentina,ARG,2020-12-30,<NA>,<NA>,<NA>,NaN,15656,NaN,NaN,NaN,346,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
2,Argentina,ARG,2020-12-31,32013,<NA>,<NA>,NaN,15656,0.07,NaN,NaN,346,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
3,Argentina,ARG,2021-01-01,<NA>,<NA>,<NA>,NaN,11070,NaN,NaN,NaN,245,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
4,Argentina,ARG,2021-01-02,<NA>,<NA>,<NA>,NaN,8776,NaN,NaN,NaN,194,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...


In [8]:
df.loc[df.source_website == 'Kuwait', ['source_website']]
df.dtypes

country                                 object
iso_code                                object
date                                    object
total_vaccinations                       Int64
people_vaccinated                        Int64
people_fully_vaccinated                  Int64
daily_vaccinations_raw                 float64
daily_vaccinations                       Int64
total_vaccinations_per_hundred         float64
people_vaccinated_per_hundred          float64
people_fully_vaccinated_per_hundred    float64
daily_vaccinations_per_million           Int64
vaccines                                object
source_name                             object
source_website                          object
dtype: object

### Transformar antes de almacenar en mysql.

In [12]:
u = df.select_dtypes(object)
df[u.columns] = u.apply(
    lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

### Load data transformada a mysql.

In [13]:
create_table_query = pd.io.sql.get_schema(df.reset_index(), 'test.vaccinations_per_country', con=engine)
print(create_table_query)
type(create_table_query)


CREATE TABLE `test.vaccinations_per_country` (
	`index` BIGINT, 
	country TEXT, 
	iso_code TEXT, 
	date TEXT, 
	total_vaccinations BIGINT, 
	people_vaccinated BIGINT, 
	people_fully_vaccinated BIGINT, 
	daily_vaccinations_raw FLOAT(53), 
	daily_vaccinations BIGINT, 
	total_vaccinations_per_hundred FLOAT(53), 
	people_vaccinated_per_hundred FLOAT(53), 
	people_fully_vaccinated_per_hundred FLOAT(53), 
	daily_vaccinations_per_million BIGINT, 
	vaccines TEXT, 
	source_name TEXT, 
	source_website TEXT
)




str

In [21]:
with open("init.sql", "a") as text_file:
    text_file.write("\n" + create_table_query)
    
with open("vaccinations_per_country.sql", "w") as text_file:
    text_file.write(create_table_query)

In [15]:
from sqlalchemy import *

if not engine.dialect.has_table(engine, "test.vaccinations_per_country"):  # If table don't exist, Create.
    print("test.vaccinations_per_country no existe")
    #metadata = MetaData(engine)
    # Create a table with the appropriate Columns
    df.to_sql('vaccinations_per_country', con=engine, schema="test", if_exists='append')
    # Implement the creation
    #metadata.create_all()

2021-02-03 03:05:51,445 INFO sqlalchemy.engine.base.OptionEngine DESCRIBE `test.vaccinations_per_country`
2021-02-03 03:05:51,448 INFO sqlalchemy.engine.base.OptionEngine {}
2021-02-03 03:05:51,451 INFO sqlalchemy.engine.base.OptionEngine ROLLBACK
test.vaccinations_per_country no existe
2021-02-03 03:05:51,465 INFO sqlalchemy.engine.base.Engine DESCRIBE `test`.`vaccinations_per_country`
2021-02-03 03:05:51,466 INFO sqlalchemy.engine.base.Engine {}
2021-02-03 03:05:51,478 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-03 03:05:51,531 INFO sqlalchemy.engine.base.Engine INSERT INTO test.vaccinations_per_country (`index`, country, iso_code, date, total_vaccinations, people_vaccinated, people_fully_vaccinated, daily_vaccinations_raw, daily_vaccinations, total_vaccinations_per_hundred, people_vaccinated_per_hundred, people_fully_vaccinated_per_hundred, daily_vaccinations_per_million, vaccines, source_name, source_website) VALUES (%(index)s, %(country)s, %(iso_code)s, %(date)s, %

In [17]:
with engine.begin() as transaction:
    transaction.execute('DELETE FROM test.vaccinations_per_country')
    df.to_sql('vaccinations_per_country', con=transaction, schema='test', if_exists='append', index=False)

2021-02-03 03:06:44,434 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-03 03:06:44,439 INFO sqlalchemy.engine.base.Engine DELETE FROM test.vaccinations_per_country
2021-02-03 03:06:44,443 INFO sqlalchemy.engine.base.Engine {}
2021-02-03 03:06:44,486 INFO sqlalchemy.engine.base.Engine DESCRIBE `test`.`vaccinations_per_country`
2021-02-03 03:06:44,487 INFO sqlalchemy.engine.base.Engine {}
2021-02-03 03:06:44,554 INFO sqlalchemy.engine.base.Engine INSERT INTO test.vaccinations_per_country (country, iso_code, date, total_vaccinations, people_vaccinated, people_fully_vaccinated, daily_vaccinations_raw, daily_vaccinations, total_vaccinations_per_hundred, people_vaccinated_per_hundred, people_fully_vaccinated_per_hundred, daily_vaccinations_per_million, vaccines, source_name, source_website) VALUES (%(country)s, %(iso_code)s, %(date)s, %(total_vaccinations)s, %(people_vaccinated)s, %(people_fully_vaccinated)s, %(daily_vaccinations_raw)s, %(daily_vaccinations)s, %(total_vaccinatio